<a href="https://colab.research.google.com/github/Sreeshbk/natural_language_process_with_tranformers/blob/main/notebooks/08_makingtransformeffecientinprod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers Efficient in Production

4 techniques
- Knowledge Distillation
- quantization
- pruning
- graph optimization

In [1]:
!pip install transformers torch xformers datasets transformers[torch] accelerate>=0.20.1 optuna onnx

In [2]:
from transformers import pipeline
bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)

In [3]:
query="""Hey, i'd like to rent a vehicle from Nov 1st to Nov 15th in Paris and I need a 15 passenger van"""
pipe(query)

[{'label': 'car_rental', 'score': 0.5490034222602844}]

In [4]:
from pathlib import Path
from time import perf_counter
import numpy as np
class PerformanceBenchmark:
  def __init__(self, pipeline, dataset, optim_type="BERT baseline"):
    self.pipeline = pipeline
    self.dataset = dataset
    self.optim_type = optim_type

  def compute_accuracy(self):
    """This overides the Performance benchmark.compute_accuracy() Method"""
    preds, labels = [], []
    for example in self.dataset:
      pred = self.pipeline(example["text"])[0]["label"]
      label = example["intent"]
      preds.append(intents.str2int(pred))
      labels.append(label)
    accuracy = accuracy_score.compute(predictions=preds, references=labels)
    print(f"Accuracy on test set -{accuracy['accuracy']:.3f}")
    return accuracy

  def compute_size(self):
    state_dict = self.pipeline.model.state_dict()
    tmp_path=Path("model.pt")
    torch.save(state_dict, tmp_path)
    size_mb = Path(tmp_path).stat().st_size/(1024*1024)
    tmp_path.unlink()
    print(f"Model size(MB) - {size_mb:.2f}")
    return {"size_mb":size_mb}

  def time_pipeline(self, query="What is the pin number for my account?"):
    latencies = []
    for _ in range(10):
      _ = self.pipeline(query)

    for _ in range(100):
      start_time = perf_counter()
      _ = self.pipeline(query)
      latency = perf_counter() - start_time
      latencies.append(latency)

    time_avg_ms = 1000 * np.mean(latencies)
    time_std_ms = 1000 * np.std(latencies)
    print(f"""Average Latency (ms) -  {time_avg_ms:.2f} +/- {time_std_ms:.2f}""")
    return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}


  def run_benchmark(self):
    metrics = {}
    metrics[self.optim_type] = self.compute_size()
    metrics[self.optim_type].update(self.time_pipeline())
    metrics[self.optim_type].update(self.compute_accuracy())
    return metrics

In [5]:
from datasets import load_dataset
clinc = load_dataset("clinc_oos","plus")

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

Dataset clinc_oos downloaded and prepared to /root/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
sample =  clinc["test"][42]
sample

{'text': 'transfer $100 from my checking to saving account', 'intent': 133}

In [7]:
intents = clinc["test"].features["intent"]
intents.int2str(sample["intent"])

'transfer'

In [8]:
from datasets import load_metric
accuracy_score = load_metric("accuracy")

<ipython-input-8-913c3a7b86a3>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_score = load_metric("accuracy")


In [9]:
list(pipe.model.state_dict().items())[42]

('bert.encoder.layer.2.attention.self.value.weight',
 tensor([[-1.0526e-02, -3.2215e-02,  2.2097e-02,  ..., -6.0953e-03,
           4.6521e-03,  2.9844e-02],
         [-1.4964e-02, -1.0915e-02,  5.2396e-04,  ...,  3.2047e-05,
          -2.6890e-02, -2.1943e-02],
         [-2.9640e-02, -3.7842e-03, -1.2582e-02,  ..., -1.0917e-02,
           3.1152e-02, -9.7786e-03],
         ...,
         [-1.5116e-02, -3.3226e-02,  4.2063e-02,  ..., -5.2652e-03,
           1.1093e-02,  2.9703e-03],
         [-3.6809e-02,  5.6848e-02, -2.6544e-02,  ..., -4.0114e-02,
           6.7487e-03,  1.0511e-03],
         [-2.4961e-02,  1.4747e-03, -5.4271e-02,  ...,  2.0004e-02,
           2.3981e-02, -4.2880e-02]]))

In [10]:
import torch
torch.save(pipe.model.state_dict(), "model.pt")

In [11]:
pb = PerformanceBenchmark(pipe, clinc["test"])
perf_metric = pb.run_benchmark()

Model size(MB) - 418.15
Average Latency (ms) -  211.89 +/- 89.16
Accuracy on test set -0.867


## Knowledge Distillation :
Training a smaller student model to mimic the behavior of a slower, larger, but better-performing teacher

In [12]:
from transformers import TrainingArguments
class DistillationTrainingArguments(TrainingArguments):
  def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
    super().__init__(*args,**kwargs)
    self.alpha = alpha
    self.temperature = temperature

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

class DistillationTrainer(Trainer):
  def __init__(self, *args, teacher_model=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.teacher_model = teacher_model

  def compute_loss(self, model, inputs, return_outputs=False):
    outputs_stu = model(**inputs)
    loss_ce = outputs_stu.loss
    logits_stu = outputs_stu.logits
    with torch.no_grad():
      outputs_tea = self.teacher_model(**inputs)
      logits_tea = outputs_tea.logits
    loss_fct = nn.KLDivLoss(reduction="batchmean")
    loss_kd = self.args.temperature ** 2* loss_fct(
        F.log_softmax(logits_stu/self.args.temperature, dim=-1),
        F.softmax(logits_tea/self.args.temperature, dim=-1)
    )
    loss = self.args.alpha *loss_ce + (1. - self.args.alpha)*loss_kd
    return (loss, outputs_stu) if return_outputs else loss

In [14]:
from transformers import AutoTokenizer
student_ckpt = "distilbert-base-uncased"
student_tokenizer = AutoTokenizer.from_pretrained(student_ckpt)

def tokenize_text(batch):
  return student_tokenizer(batch["text"], truncation=True)

clinc_enc = clinc.map(tokenize_text, batched=True, remove_columns=["text"])
clinc_enc = clinc_enc.rename_column("intent","labels")

Map:   0%|          | 0/15250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_score.compute(predictions=predictions, references=labels)

In [17]:
batch_size = 48
finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"
student_training_args = DistillationTrainingArguments(
    output_dir=finetuned_ckpt,
    learning_rate=2e-5,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    alpha=1, weight_decay=0.01, push_to_hub=True
)

In [18]:
id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [19]:
from transformers import AutoConfig

num_labels = intents.num_classes
student_config = (AutoConfig.from_pretrained(
    student_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id
))

In [20]:
import torch
from transformers import AutoModelForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def student_init():
  return (AutoModelForSequenceClassification.from_pretrained(student_ckpt, config=student_config).to(device))

In [21]:
teacher_ckpt = "transformersbook/distilbert-base-uncased-finetuned-clinc"
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_ckpt, num_labels=num_labels)
distilbert_trainer = DistillationTrainer(model_init=student_init,
    teacher_model=teacher_model,
    args=student_training_args,
    train_dataset=clinc_enc['train'],
    eval_dataset=clinc_enc['validation'],
    compute_metrics=compute_metrics,
    tokenizer=student_tokenizer
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.we

LocalTokenNotFoundError: ignored

In [ ]:
distilbert_trainer.train()

In [ ]:
def objective(trial):
  x = trial.suggest_float("x", -2, 2)
  y = trial.suggest_float("y", -2, 2)
  return (1-x)**2+100*(y-x**2)**2

In [ ]:
import optuna
study = optuna.create_study()
study.optimize(objective, n_trials=1000)

In [ ]:
study.best_params

In [ ]:
def hp_space(trial):
  return {
      "num_train_epochs" : trial.suggest_int("num_train_epochs", 5 , 10),
      "alpha" : trial.suggest_float("alpha", 0, 1),
      "temperature" : trial.suggest_int("temperature", 2, 20)
  }

In [ ]:
best_run = distilbert_trainer.hyperparameter_search(
    n_trials = 20, direction="maximize", hp_space = hp_space
)

In [ ]:
print(best_run)

In [ ]:
for k,v in best_run.hyperparameters.items():
  setattr(student_training_args, k, v)

In [ ]:
distilled_ckpt = "distilbert-base-uncased-distilled-clinc"
student_training_args.output_dir = distilled_ckpt
distilbert_trainer = DistillationTrainer(model_init=student_init,
    teacher_model=teacher_model,
    args=student_training_args,
    train_dataset=clinc_enc['train'],
    eval_dataset=clinc_enc['validation'],
    compute_metrics=compute_metrics,
    tokenizer=student_tokenizer
)

In [ ]:
distilbert_trainer.train()

In [ ]:
distilbert_trainer.push_to_hub("Training Complete")

In [ ]:
distilled_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
pipe = pipeline("text-classification", model=distilled_ckpt)
optim_type = "Distillation"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type = optim_type)
perf_metric.update(pb.run_benchmark())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_metrics(perf_metrics, current_optim_type):
    df = pd.DataFrame.from_dict(perf_metrics, orient='index')

    for idx in df.index:
        df_opt = df.loc[idx]
        # Add a dashed circle around the current optimization type
        if idx == current_optim_type:
            plt.scatter(df_opt["time_avg_ms"], df_opt["accuracy"] * 100,
                        alpha=0.5, s=df_opt["size_mb"], label=idx,
                        )
        else:
            plt.scatter(df_opt["time_avg_ms"], df_opt["accuracy"] * 100,
                        s=df_opt["size_mb"], label=idx, alpha=0.5)

    legend = plt.legend(bbox_to_anchor=(1,1))
    for handle in legend.legendHandles:
        handle.set_sizes([20])

    plt.ylim(80,90)
    # Use the slowest model to define the x-axis range
    xlim = int(perf_metrics["BERT baseline"]["time_avg_ms"] + 3)
    plt.xlim(1, xlim)
    plt.ylabel("Accuracy (%)")
    plt.xlabel("Average latency (ms)")
    plt.show()

plot_metrics(perf_metric, optim_type)

In [ ]:
state_dict = pipe.model.state_dict()
weights = state_dict["distilbert.transformer.layer.0.attention.out_lin.weight"]
plt.hist(weights.flatten().numpy(),bins=250, range=(-0.3,0.3), edgecolor="C0")
plt.show()

In [ ]:
zero_point = 0
scale = (weights.max() - weights.min())/(127-(-128))

In [ ]:
(weights/(scale +zero_point)).clamp(-128,127).round().char()

In [ ]:
from torch import quantize_per_tensor
dtype = torch.qint8
quantized_weights = quantize_per_tensor(weights, scale, zero_point, dtype)
quantized_weights.int_repr()

In [ ]:
plt.hist(quantized_weights.int_repr().flatten().numpy(),bins=250, range=(-0.3,0.3), edgecolor="C0")
plt.show()

In [ ]:
%%timeit
weights @ weights

In [ ]:
from torch.nn.quantized import QFunctional
q_fn =QFunctional()

In [ ]:
%%timeit
q_fn.mul(quantized_weights, quantized_weights)

In [ ]:
import sys
sys.getsizeof(weights.storage())/sys.getsizeof(quantized_weights.storage())

In [ ]:
from torch.quantization import quantize_dynamic
model_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt).to("cpu"))

In [ ]:
model_quantized = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

In [ ]:
pipe = pipeline("text-classification", model= model_quantized, tokenizer=tokenizer)
optim_type = "Distillation + quantization"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metric.update(pb.run_benchmark())

In [ ]:
plot_metrics(perf_metric, optim_type)